In [36]:
import os
import sqlalchemy
import requests
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize

## Extract

### From API of World Bank

#### Rural population

In [2]:

url = "https://api.worldbank.org/v2/country/all/indicator/SP.RUR.TOTL?format=json&per_page=20000"
response = requests.get(url)
data_rural = response.json()[1]  # The data is in the second element of the response list


rural_population_df = pd.json_normalize(data_rural)


#### Access to electricity (% of population)

In [3]:

url = "https://api.worldbank.org/v2/country/all/indicator/EG.ELC.ACCS.ZS?format=json&per_page=20000"
response = requests.get(url)
data_electricity = response.json()[1]  # The data is in the second element of the response list


access_electricity_df = pd.json_normalize(data_electricity)

### Reshape of json file Rural Population

#### Drop columns

In [4]:
rural_population_df.drop(['unit', 'obs_status', 'decimal','indicator.id','country.id'], axis=1, inplace=True)


#### Convert types

In [5]:
rural_population_df['country.value'] = rural_population_df['country.value'].astype('string')
rural_population_df['countryiso3code'] = rural_population_df['countryiso3code'].astype('string')

In [6]:
rural_population_df

,countryiso3code,date,value,indicator.value,country.value
0,AFE,2021,440599883.0,Rural population,Africa Eastern and Southern
1,AFE,2020,432797335.0,Rural population,Africa Eastern and Southern
2,AFE,2019,424791683.0,Rural population,Africa Eastern and Southern
3,AFE,2018,416834659.0,Rural population,Africa Eastern and Southern
4,AFE,2017,409014178.0,Rural population,Africa Eastern and Southern
...,...,...,...,...,...
16487,ZWE,1964,3702920.0,Rural population,Zimbabwe
16488,ZWE,1963,3610652.0,Rural population,Zimbabwe
16489,ZWE,1962,3519986.0,Rural population,Zimbabwe
16490,ZWE,1961,3422606.0,Rural population,Zimbabwe


#### Pivot table to Reshape data

In [7]:
rural_population_df = pd.pivot_table(rural_population_df, values='value', index=['country.value','countryiso3code','indicator.value'], columns='date')
rural_population_df.head(5)

,,date,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
country.value,countryiso3code,indicator.value,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,Rural population,7898093.0,8026804.0,8163985.0,8308019.0,8458694.0,8617815.0,8782038.0,8951970.0,9128927.0,9311235.0,...,23105778.0,23853670.0,24672275.0,25381619.0,25970228.0,26643455.0,27333488.0,28042342.0,28829317.0,29547690.0
Africa Eastern and Southern,AFE,Rural population,111658758.0,114296883.0,117064799.0,119893122.0,122832934.0,125871499.0,128929776.0,132073909.0,135349905.0,138711770.0,...,369971909.0,377783210.0,385577486.0,393451859.0,401294002.0,409014178.0,416834659.0,424791683.0,432797335.0,440599883.0
Africa Western and Central,AFW,Rural population,82954373.0,84327418.0,85737206.0,87186879.0,88657553.0,90158852.0,91732332.0,93316168.0,94927194.0,96574557.0,...,215054125.0,218561121.0,222082250.0,225573122.0,229093774.0,232644696.0,236154623.0,239621856.0,243081107.0,246481855.0
Albania,ALB,Rural population,1114818.0,1146208.0,1180553.0,1214693.0,1248887.0,1282417.0,1315273.0,1348911.0,1386348.0,1424962.0,...,1324613.0,1291587.0,1258985.0,1226200.0,1195854.0,1167112.0,1137407.0,1106598.0,1075204.0,1041188.0
Algeria,DZA,Rural population,7917904.0,7910599.0,7866229.0,7820600.0,7777451.0,7720580.0,7714349.0,7866724.0,8024202.0,8184460.0,...,11582446.0,11561310.0,11542390.0,11527620.0,11513248.0,11496842.0,11475841.0,11449736.0,11413449.0,11370967.0


In [8]:
rural_population_df =  rural_population_df.reset_index()

In [9]:
rural_population_df

date,country.value,countryiso3code,indicator.value,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afghanistan,AFG,Rural population,7.898093e+06,8.026804e+06,8.163985e+06,8.308019e+06,8.458694e+06,8.617815e+06,8.782038e+06,...,2.310578e+07,2.385367e+07,2.467228e+07,2.538162e+07,2.597023e+07,2.664346e+07,2.733349e+07,2.804234e+07,2.882932e+07,2.954769e+07
1,Africa Eastern and Southern,AFE,Rural population,1.116588e+08,1.142969e+08,1.170648e+08,1.198931e+08,1.228329e+08,1.258715e+08,1.289298e+08,...,3.699719e+08,3.777832e+08,3.855775e+08,3.934519e+08,4.012940e+08,4.090142e+08,4.168347e+08,4.247917e+08,4.327973e+08,4.405999e+08
2,Africa Western and Central,AFW,Rural population,8.295437e+07,8.432742e+07,8.573721e+07,8.718688e+07,8.865755e+07,9.015885e+07,9.173233e+07,...,2.150541e+08,2.185611e+08,2.220822e+08,2.255731e+08,2.290938e+08,2.326447e+08,2.361546e+08,2.396219e+08,2.430811e+08,2.464819e+08
3,Albania,ALB,Rural population,1.114818e+06,1.146208e+06,1.180553e+06,1.214693e+06,1.248887e+06,1.282417e+06,1.315273e+06,...,1.324613e+06,1.291587e+06,1.258985e+06,1.226200e+06,1.195854e+06,1.167112e+06,1.137407e+06,1.106598e+06,1.075204e+06,1.041188e+06
4,Algeria,DZA,Rural population,7.917904e+06,7.910599e+06,7.866229e+06,7.820600e+06,7.777451e+06,7.720580e+06,7.714349e+06,...,1.158245e+07,1.156131e+07,1.154239e+07,1.152762e+07,1.151325e+07,1.149684e+07,1.147584e+07,1.144974e+07,1.141345e+07,1.137097e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,West Bank and Gaza,PSE,Rural population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.010163e+06,1.024762e+06,1.038634e+06,1.051809e+06,1.064347e+06,1.073875e+06,1.089088e+06,1.103858e+06,1.118249e+06,1.132085e+06
259,World,WLD,Rural population,2.012267e+09,2.024481e+09,2.046843e+09,2.078385e+09,2.109469e+09,2.146075e+09,2.184623e+09,...,3.393623e+09,3.403214e+09,3.411423e+09,3.417903e+09,3.423737e+09,3.428913e+09,3.432505e+09,3.434599e+09,3.435444e+09,3.432438e+09
260,"Yemen, Rep.",YEM,Rural population,5.038095e+06,5.112550e+06,5.187771e+06,5.261519e+06,5.339744e+06,5.425010e+06,5.515212e+06,...,1.758069e+07,1.792844e+07,1.827139e+07,1.859935e+07,1.891276e+07,1.921720e+07,1.950825e+07,1.978829e+07,2.004581e+07,2.026854e+07
261,Zambia,ZMB,Rural population,2.553409e+06,2.609333e+06,2.665887e+06,2.720673e+06,2.762825e+06,2.803071e+06,2.841231e+06,...,8.794599e+06,9.009203e+06,9.225180e+06,9.439084e+06,9.651859e+06,9.864042e+06,1.007353e+07,1.027983e+07,1.048046e+07,1.067283e+07


### Reshape of json file Access to Electricity

In [10]:
access_electricity_df

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2021,NaN,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZH,Africa Eastern and Southern
1,AFE,2020,45.609604,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZH,Africa Eastern and Southern
2,AFE,2019,44.073912,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZH,Africa Eastern and Southern
3,AFE,2018,42.880977,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZH,Africa Eastern and Southern
4,AFE,2017,40.092163,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
16487,ZWE,1964,NaN,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZW,Zimbabwe
16488,ZWE,1963,NaN,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZW,Zimbabwe
16489,ZWE,1962,NaN,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZW,Zimbabwe
16490,ZWE,1961,NaN,,,1,EG.ELC.ACCS.ZS,Access to electricity (% of population),ZW,Zimbabwe


#### Drop columns

In [11]:
access_electricity_df.drop(['unit', 'obs_status', 'decimal','indicator.id','country.id'], axis=1, inplace=True)

#### Convert types

In [12]:
access_electricity_df['country.value'] = access_electricity_df['country.value'].astype('string')
access_electricity_df['countryiso3code'] = access_electricity_df['countryiso3code'].astype('string')

#### Pivot table to Reshape data

In [13]:
access_electricity_df = pd.pivot_table(access_electricity_df, values='value', index=['country.value','countryiso3code','indicator.value'], columns='date')
access_electricity_df.head(5)

,,date,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country.value,countryiso3code,indicator.value,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.222019,69.099998,68.290649,89.500000,71.500000,97.699997,97.699997,96.616135,97.699997,97.699997
Africa Eastern and Southern,AFE,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.948628,31.682318,31.610692,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604
Africa Western and Central,AFW,Access to electricity (% of population),NaN,NaN,NaN,31.57513,32.609692,33.605668,32.629322,32.461797,33.45863,34.810428,...,45.896347,44.137410,47.029335,47.587145,46.750946,50.931399,48.835062,51.238518,51.260220,52.082053
Albania,ALB,Access to electricity (% of population),100.0,100.0,100.0,100.00000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,...,100.000000,99.900002,100.000000,99.949997,99.980003,99.889999,99.889999,100.000000,100.000000,100.000000
Algeria,DZA,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.888031,98.764664,99.002205,99.087013,99.186661,99.350250,99.635490,99.697838,99.500000,99.804131


In [14]:
access_electricity_df = access_electricity_df.reset_index()

#### Drop columns in a range of 1960 to 1989 from the rural_population

In [15]:
drop_cols = [str(year) for year in range(1960, 2000)]
drop_cols= [col for col in rural_population_df.columns if any(year in col for year in drop_cols)]
rural_population_df.drop(columns=drop_cols, inplace=True)


### Merge of the two dataframes (rural_population_df, access_electricity_df)

In [16]:
main_df = rural_population_df.merge(access_electricity_df, on='countryiso3code', how='left')

#### Drop columns

In [17]:
main_df.drop(['country.value_y','2021'], axis=1, inplace=True)

#### Re order the columns 

In [18]:
list=['country.value_x','countryiso3code','indicator.value_x','indicator.value_y']

for year in range(2000, 2021):
  if year:
    list.append(str(year) + "_x")
    list.append(str(year) + "_y")
 

In [19]:
main_df = main_df.reindex(columns=list)


#### Get the number of missing data points per column

In [20]:
missing_values = main_df.isnull().sum()

In [21]:
missing_values

date
country.value_x      0
countryiso3code      0
indicator.value_x    0
indicator.value_y    1
2000_x               0
2000_y               5
2001_x               0
2001_y               5
2002_x               0
2002_y               4
2003_x               0
2003_y               4
2004_x               0
2004_y               4
2005_x               0
2005_y               4
2006_x               0
2006_y               3
2007_x               0
2007_y               2
2008_x               0
2008_y               2
2009_x               0
2009_y               1
2010_x               0
2010_y               1
2011_x               0
2011_y               1
2012_x               0
2012_y               1
2013_x               0
2013_y               1
2014_x               0
2014_y               1
2015_x               0
2015_y               1
2016_x               0
2016_y               1
2017_x               0
2017_y               1
2018_x               0
2018_y               1
2019_x               0
2019_y

#### Percentage of missing data in the main_df

In [22]:
total_cells = np.product(main_df.shape)
total_missing = missing_values.sum()

percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

0.36363636363636365


#### Drop the rows that have NaN value in the column 'countryiso3code'

In [23]:
main_df.drop(main_df['countryiso3code'][main_df["countryiso3code"].str.strip(" ") == ""].index, inplace=True)

#### Data imputacion of missing values of categorical column indicator.value_y

In [24]:
main_df['indicator.value_y'].fillna(main_df['indicator.value_y'].mode()[0], inplace=True)

#### Fill missing values with zeros

In [25]:
main_df.fillna(0, inplace=True)

#### Load of the data to a POSTGRESQL database

In [ ]:
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_host = os.environ['DB_HOST']
db_port = os.environ['DB_PORT']
db_name = os.environ['DB_NAME']

In [35]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [ ]:
main_df.to_sql('mergeindicators', engine)